# Test book

In [1]:
from scipy.io import loadmat
import mat73
import os
import numpy as np
import pandas as pd
from GLM import glm


In [ ]:
data=loadmat('/Users/user/PycharmProjects/PacManMain/GLM/exampleData/PMD.mat')
data=pd.DataFrame(data['data'][0])
psth=np.array(data['psth'][0])

column_names = [f'psth_{i}' for i in range(psth.shape[1])]

data = data.applymap(lambda x: [item[0] for item in x])
data = data.apply(pd.Series.explode).reset_index(drop=True)
data['rewardVal']=data.rewardVal.astype('category')
data=data[data['rewardVal']!=0]

#Add in psth
for i in range(psth.shape[1]):
    data[f'psth_{i}'] = psth[:, i]


X=data[['self_spd','prey_spd','prey_dist','rewardVal']]
X['prey_dist'] = pd.to_numeric(X['prey_dist'], errors='coerce')
X['prey_spd'] = pd.to_numeric(X['prey_spd'], errors='coerce')
X['self_spd'] = pd.to_numeric(X['self_spd'], errors='coerce')

y = data['psth_49']

#Make formulas
formulas=[]
formulas.append('y ~ (cr(self_spd,df=5)+ cr(prey_spd, df=5)+ cr(prey_dist, df=5))*C(rewardVal)-1')
formulas.append('y ~ (cr(self_spd,df=5)+ cr(prey_spd, df=5))*C(rewardVal)-1')
formulas.append('y ~ cr(self_spd,df=5) + cr(prey_spd, df=5)+C(rewardVal) -1')
formulas.append('y ~ cr(prey_dist,df=5)*C(rewardVal)-1')
formulas.append('y ~ cr(self_spd,df=5)*C(rewardVal)-1')
formulas.append('y ~ cr(prey_spd,df=5)*C(rewardVal)-1')
formulas.append('y ~ cr(self_spd,df=5)+C(rewardVal)-1')
formulas.append('y ~ cr(prey_spd,df=5)+C(rewardVal)-1')
formulas.append('y ~ cr(prey_dist,df=5)-1')
formulas.append('y ~ cr(self_spd,df=5)')
formulas.append('y ~ cr(prey_spd,df=5)')
formulas.append('y ~ C(rewardVal)-1')
formulas.append('y ~1')

mod=glm.PoissonGLM()
mod.add_data(X,y).make_preprocessor(formulas=formulas,metric='cv',l2reg=0.001).fit(params={'cv': 5, 'shuffleTime': False})


x1 = np.median(mod.X['prey_spd'])
x2= np.median(mod.X['self_spd'])
x1=np.linspace((mod.X['self_spd']).min(),(mod.X['self_spd']).max(),100)
levels = [1, 2,3,4,5] 

# Create a DataFrame to hold the predictions for each level of the categorical variable
pred_data = pd.DataFrame({
    'self_spd':np.repeat(x1,len(levels))
    'rewardVal': np.tile(levels, len(x1))
})

mod.predict(pred_data)

        # x2_median = data['x2'].median()  # Median value for x2
        # levels = ['A', 'B', 'C']  # Levels for categorical variable
# pred_data = pd.DataFrame({'rewardVal':np.tile(levels,1)})


# 
# levels = ['A', 'B']  # Levels for categorical variable
# 
# # Create a DataFrame to hold the predictions for each level of the categorical variable
# pred_data = pd.DataFrame({
#     'x1':x1_median,
#     'x2': np.repeat(x2_range,len(levels)),
#     'cat_var': np.tile(levels, len(x2_range))
# })

Generate data

Make predicted set

In [ ]:
data=loadmat('/Users/user/PycharmProjects/PacManMain/GLM/exampleData/data4Justin.mat')
data=pd.DataFrame(data['data'][0])
data = data.applymap(lambda x: [item[0] for item in x])
data = data.apply(pd.Series.explode).reset_index(drop=True)
rewardVal=data.rewardVal

data=loadmat('/Users/user/PycharmProjects/PacManMain/GLM/exampleData/data4Justin2.mat')

data=pd.DataFrame(data['data'][0])
psth=np.array(data['accspks'][0])

column_names = [f'psth_{i}' for i in range(psth.shape[1])]

data = data.applymap(lambda x: [item[0] for item in x])
data = data.apply(pd.Series.explode).reset_index(drop=True)
data['rewardVal']=rewardVal
data['rewardVal']=data.rewardVal.astype('category')



#Add in psth
for i in range(psth.shape[1]):
    data[f'psth_{i}'] = psth[:, i]

data=data[data['rewardVal']!=0]
# data['rewardVal']=data.rewardVal.astype('int')


X=data[['self_spd','prey_spd','rewardVal']]
X['prey_spd'] = pd.to_numeric(X['prey_spd'], errors='coerce')
X['self_spd'] = pd.to_numeric(X['self_spd'], errors='coerce')
X['prey_spd']=(X['prey_spd']/10)
X['self_spd']=(X['self_spd']/10)

y = data['psth_1']

#Make formulas
formulas=[]
formulas.append('y ~ cr(self_spd,df=5)*cr(prey_spd, df=5)-1')

formulas.append('y ~ cr(self_spd,df=5)+ cr(prey_spd, df=5)-1')
formulas.append('y ~ cr(self_spd,df=5)-1')
formulas.append('y ~ cr(prey_spd,df=5)-1')
formulas.append('y ~1')

mod=glm.PoissonGLM()
mod.add_data(X,y).make_preprocessor(formulas=formulas,metric='cv',l2reg=0.001).fit(params={'cv': 5, 'shuffleTime': False})


x1=np.linspace((mod.X['prey_spd']).min(),(mod.X['prey_spd']).max(),100)
x2=np.linspace((mod.X['self_spd']).min(),(mod.X['self_spd']).max(),100)

pred_data = pd.DataFrame({
        'prey_spd':x1,
        'self_spd':np.median((mod.X['self_spd']))})
# Create a DataFrame to hold the predictions for each level of the categorical variable
predictions=np.zeros((100,100))
for i in range(100):
    pred_data = pd.DataFrame({
        'prey_spd':x1,
        'self_spd':x2[i]
    })

    mod.predict(pred_data)
    predictions[:,i]=np.array(mod.predicted_y)
